In [8]:
from datasets import load_dataset
import os
DATA_DIR = "/Users/juanigp/Desktop/data/e-gmd-v1.0.0"
csv_dir = "e-gmd-v1.0.0.csv"
csv_dir = os.path.join(DATA_DIR, csv_dir)

In [11]:
dataset = load_dataset('csv', data_files=csv_dir)
dataset = dataset['train']

Using custom data configuration default-51ad44df56849337
Reusing dataset csv (/Users/juanigp/.cache/huggingface/datasets/csv/default-51ad44df56849337/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 174.82it/s]


In [12]:
dataset[0]

{'drummer': 'drummer1',
 'session': 'drummer1/eval_session',
 'id': 'drummer1/eval_session/1',
 'style': 'funk/groove1',
 'bpm': 138,
 'beat_type': 'beat',
 'time_signature': '4-4',
 'duration': 27.826099773242632,
 'split': 'test',
 'midi_filename': 'drummer1/eval_session/1_funk-groove1_138_beat_4-4_1.midi',
 'audio_filename': 'drummer1/eval_session/1_funk-groove1_138_beat_4-4_1.wav',
 'kit_name': 'Acoustic Kit'}

## Notas:

#### Pipeline
- MidiTOK: Tokenizers have a .vocab member to get the vocabulary; and .tokenize_midi_dataset that can tokenize a list of midi paths and return a json
- huggingface docs - datasets- how tos - process, has examples of functional transforms. Maybe this way it is more intuitive or convenient.

#### Resources:
- https://huggingface.co/docs/datasets/loading


#### Tasks:
- Autoregressive: first to try
- conditioned generation: can add the labels to the vocabulary, and prepend these tokens to the sequences
- see if i need remove some tokens, for example velocity.


In [15]:
from miditoolkit import MidiFile
from miditok import REMI
tokenizer = REMI()

def map_func(x, data_dir, tokenizer):
    x['midi_filename'] = os.path.join(data_dir, x['midi_filename'])
    midi_file = MidiFile(x['midi_filename'])
    tokens = tokenizer.midi_to_tokens(midi_file)[0]
    x['input_ids'] = tokens
    return x

In [21]:
dataset = dataset.map(lambda x: map_func(x, DATA_DIR, tokenizer), num_proc=8)

Loading cached processed dataset at /Users/juanigp/.cache/huggingface/datasets/csv/default-51ad44df56849337/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-2d69005c3bc045e4.arrow
Loading cached processed dataset at /Users/juanigp/.cache/huggingface/datasets/csv/default-51ad44df56849337/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-d8571d4672a6fe72.arrow
Loading cached processed dataset at /Users/juanigp/.cache/huggingface/datasets/csv/default-51ad44df56849337/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-836c9eacec353afe.arrow
Loading cached processed dataset at /Users/juanigp/.cache/huggingface/datasets/csv/default-51ad44df56849337/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-78045bc30a1b1de2.arrow
Loading cached processed dataset at /Users/juanigp/.cache/huggingface/datasets/csv/default-51ad44df56849337/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb

In [23]:
dataset = dataset.train_test_split(test_size = 0.1)

In [25]:
import sys
sys.getsizeof(dataset)

248

In [28]:
dataset['train'][0]['input_ids']

[1,
 186,
 17,
 121,
 123,
 32,
 121,
 123,
 190,
 32,
 97,
 123,
 194,
 19,
 121,
 123,
 195,
 32,
 121,
 122,
 198,
 32,
 104,
 123,
 200,
 19,
 121,
 123,
 202,
 32,
 109,
 123,
 204,
 19,
 121,
 123,
 206,
 17,
 106,
 122,
 32,
 116,
 123,
 208,
 17,
 121,
 122,
 210,
 19,
 121,
 122,
 32,
 121,
 122,
 214,
 32,
 104,
 122,
 1,
 186,
 17,
 121,
 122,
 32,
 121,
 122,
 190,
 32,
 102,
 122,
 194,
 19,
 121,
 122,
 32,
 121,
 123,
 198,
 32,
 106,
 122,
 200,
 19,
 121,
 123,
 202,
 32,
 115,
 122]